In [2]:
import pandas as pd
import requests
import os
from unidecode import unidecode
import json

In [3]:
df = pd.read_csv("missing_gender.csv")

In [4]:
df[df['name_id'].isnull()]

,id,person,character,movie,name_id,gender
606,xst-prd-502,Read the Book,Deenie by Judy Blume,1608_Judy_Blume_Forever,NaN,NaN


In [6]:
df.head()

,id,person,character,movie,name_id,gender
0,/name/nm13818646/Self,Kenneth Scheuvront,Self,1000_On_The_Trail_of_UFOS_Dark_Sky,nm13818646,NaN
1,/name/nm10541807/Self,Shannon Legro,Self,1000_On_The_Trail_of_UFOS_Dark_Sky,nm10541807,NaN
2,/name/nm13818650/Self,Dave Roberts,Self,1000_On_The_Trail_of_UFOS_Dark_Sky,nm13818650,NaN
3,/name/nm13818652/Self,Steve Hummel,Self,1000_On_The_Trail_of_UFOS_Dark_Sky,nm13818652,NaN
4,/name/nm13818651/Self,Matt Welsch,Self,1000_On_The_Trail_of_UFOS_Dark_Sky,nm13818651,NaN


In [16]:
df = df[~df['name_id'].isnull()]

In [17]:
try:
    with open("genderize_data.json", "r") as f:
        out = json.load(f)
except FileNotFoundError:
    out = []

name_list = list(map(lambda x: x['name_id'], out))
df_sub = df[~df['name_id'].isin(name_list)]

for index, row in df_sub.iterrows():
    person = row['person']
    # person = "".join(c for c in row['person'] if c.isalpha())

    f_name = person.split()[0]

    res = requests.get(f"https://api.genderize.io?name={f_name}")
    
    content = res.json()
    if 'error' in content:
        print(content['error'])
        break
    
    content['name_id'] = row['name_id']

    print(res.status_code, index, row['name_id'], person, content['gender'])
    if res.status_code == 200:
        out.append(content)
    else:
        break

200 1397 nm14225415 Bekah Sosland Siegfriedt female
200 1398 nm0947418 Boris Yeltsin male
200 1399 nm10867191 Igor Malashenko male
200 1400 nm9717533 Arkady Ostrovsky male
200 1401 nm9723725 Igor Sechin male
200 1402 nm9543537 Henia Bryer female
200 1403 nm9543538 Maurice Bryer male
200 1404 nm13857135 Mike Bell male
200 1405 nm13857134 Mark Bell male
200 1406 nm13857131 Rosemary Bell female
200 1407 nm13857141 Richard Taite male
200 1408 nm13857144 Rand McClain male
200 1409 nm13857136 Stuart H. Garber male
200 1410 nm14167562 Tanna Leone female
200 1411 nm13857139 Gwen Olson female
200 1412 nm13857137 David Healey male
200 1413 nm7753055 Ted Lieu male
200 2364 nm11426530 Pravin Bhosale male
200 2365 nm1223910 Sujoy Ghosh male
200 2366 nm0384065 William Hill male
200 2367 nm1939381 Martin Sixsmith male
200 2368 nm2281605 Boris Berezovsky male
200 2369 nm2286394 Vladimir Gusinsky male
200 2370 nm10867193 Maria Logan female
200 2371 nm6281092 Alexei Navalny male
200 2372 nm3236749 Brock

In [19]:
len(out)

2374

In [21]:
with open("genderize_data.json", "w") as f:
    f.write(json.dumps(out))